In [1]:
# from transformers import AutoModelForCausalLM, AutoTokenizer

# model_name = "Qwen/Qwen2.5-7B-Instruct"

# model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     torch_dtype="auto",
#     device_map="auto"
# )
# tokenizer = AutoTokenizer.from_pretrained(model_name)

# prompt = "Give me a short introduction to large language model."
# messages = [
#     {"role": "system", "content": "You are Qwen, created by Alibaba Cloud. You are a helpful assistant."},
#     {"role": "user", "content": prompt}
# ]
# text = tokenizer.apply_chat_template(
#     messages,
#     tokenize=False,
#     add_generation_prompt=True
# )
# model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

# generated_ids = model.generate(
#     **model_inputs,
#     max_new_tokens=512
# )
# generated_ids = [
#     output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
# ]

# response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]


In [7]:
# import pandas as pd
# answer_df = pd.read_csv("../../data2/answer_df_qwen.csv")
# answer_df.iloc[0:700].to_csv("../../data2/answer_df_qwen1.csv", index=False)
# answer_df.iloc[700:].to_csv("../../data2/answer_df_qwen2.csv", index=False)

In [1]:
piece_id = 1
answer_df_path = "../../data2/answer_df_qwen"+str(piece_id)+".csv"

In [2]:
import pandas as pd
import numpy as np
answer_df = pd.read_csv(answer_df_path)
query_df = pd.read_csv("../../data2/fea_df.csv")

if not 'answer_string' in answer_df.columns:
    answer_df['answer_string'] = np.nan
    
print(answer_df.shape)
query_df

(700, 3)


,fea,description
0,weightbasedhealth,Judgments about health based on weight or appe...
1,weightstigma,"Negative attitudes, discrimination, or prejudi..."
2,weightbasedhealthcare,Healthcare decisions influenced by weight. Not...
3,encourageweightloss,Encouragement of weight loss
4,dietpromotion,Advocacy for diets or restrictive eating
5,thinness,Reassurance of thinness. Maintaining thinness
6,notlowweightenough,"Not sick enough, or not thin enought, or not l..."
7,weightblame,Weight blamed for health issues or concerns
8,weightvalue,Weight tied to personality characteristics


In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer


print(torch.cuda.is_available())
if torch.cuda.is_available():
    torch.cuda.empty_cache()
device = 0 if torch.cuda.is_available() else -1  # Use GPU if available, otherwise fallback to CPU

model_name = "Qwen/Qwen2.5-7B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16, #torch_dtype="auto",
    device_map="auto" if device == 0 else None,  # Automatically distribute across devices or use CPU
)
tokenizer = AutoTokenizer.from_pretrained(model_name)


True


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [4]:
system_message = """
You are an AI assistant designed to answer questions.
Use the exact answer format asked.
"""

def format_user_message(text):
    question_content = "The provided parapraph is eating disorders patients' experiences with physicians. Weight stigma is indicated by weight-or-appearance-related treatment or judgement in this context. Does the paragraph mention any of the following topics:\n"
    for i in range(len(query_df)):
        question_content += f"  ({i+1}) {query_df.fea[i]}: {query_df.description[i]}.\n"
    answer_content = "Return answer in format:\n"
    for i in range(len(query_df)): 
        answer_content += f"  ({i+1}) {query_df.fea[i]}: [yes/no], related phrases if any: \n"
    paragragh_content = f"Paragraph: '{text}' \n"
    user_message = question_content + paragragh_content + answer_content
    #print(user_message)
    
    return user_message




In [5]:
from tqdm import tqdm
for k in tqdm(range(1000), desc="Processing batch"):
    batch_size = 10

    # Filter for rows where 'answer_string' is NaN
    unanswered_df = answer_df[answer_df['answer_string'].isna()]

    # Get the indices of these NaN entries in the original DataFrame
    indices_to_update = unanswered_df.index[:batch_size]

    # Prepare prompt content for the first 10 entries with NaN answer_string
    user_messages = [format_user_message(text) for text in unanswered_df['text_w_eos'].iloc[:batch_size]]

    # Save the indices list if needed for later use
    indices_to_update_list = list(indices_to_update)

    if len(indices_to_update_list)>0:
        # query process
        messages = [ [ {"role": "system", "content": system_message}, {"role": "user", "content": user_message}] for user_message in user_messages]
        inputs = [tokenizer.apply_chat_template(
            message,
            tokenize=False,
            add_generation_prompt=True ) for message in messages]
        responses = []
        for i in range(len(inputs)):
            model_inputs = tokenizer([inputs[i]], return_tensors="pt").to(model.device)
            generated_ids = model.generate(
                **model_inputs,
                max_new_tokens=512,
                temperature=0.1
            )
            generated_ids = [
                output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
            ]
            responses.append(tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0])

        # Display and process responses in a loop
        for i, response in enumerate(responses):
            #display(Markdown(colorize_text(f"{response}")))
            # Extract answer if available
            answer = response
            # Use the original index from indices_to_update_list
            answer_df.loc[indices_to_update_list[i], 'answer_string'] = answer

        print(answer_df.loc[indices_to_update_list, ['sm_id','answer_string']])
        answer_df.to_csv(answer_df_path, index=False)
    
    else:
        break
    


Processing batch:   0%|          | 1/1000 [01:03<17:36:30, 63.45s/it]

   sm_id                                      answer_string
0   1690  (1) weightbasedhealth: no, related phrases if ...
1    724  (1) weightbasedhealth: no, related phrases if ...
2   2800  (1) weightbasedhealth: no, related phrases if ...
3   1771  (1) weightbasedhealth: no, related phrases if ...
4   1975  (1) weightbasedhealth: no, related phrases if ...
5    126  (1) weightbasedhealth: no, related phrases if ...
6   1511  (1) weightbasedhealth: no, related phrases if ...
7   4091  (1) weightbasedhealth: no, related phrases if ...
8    371  (1) weightbasedhealth: no, related phrases if ...
9   4313  (1) weightbasedhealth: no, related phrases if ...


Processing batch:   0%|          | 2/1000 [02:01<16:38:41, 60.04s/it]

    sm_id                                      answer_string
10    430  (1) weightbasedhealth: no, related phrases if ...
11     70  (1) weightbasedhealth: no, related phrases if ...
12   2225  (1) weightbasedhealth: no, related phrases if ...
13    578  (1) weightbasedhealth: no, related phrases if ...
14    913  (1) weightbasedhealth: no, related phrases if ...
15   2367  (1) weightbasedhealth: no, related phrases if ...
16   4577  (1) weightbasedhealth: no, related phrases if ...
17   2536  (1) weightbasedhealth: no, related phrases if ...
18   1085  (1) weightbasedhealth: no, related phrases if ...
19   3916  (1) weightbasedhealth: no, related phrases if ...


Processing batch:   0%|          | 3/1000 [02:57<16:10:21, 58.40s/it]

...

     sm_id                                      answer_string
670   1932  (1) weightbasedhealth: no, related phrases if ...
671   1557  (1) weightbasedhealth: no, related phrases if ...
672    799  (1) weightbasedhealth: no, related phrases if ...
673   2639  (1) weightbasedhealth: no, related phrases if ...
674   2422  (1) weightbasedhealth: no, related phrases if ...
675   2601  (1) weightbasedhealth: no, related phrases if ...
676   3127  (1) weightbasedhealth: no, related phrases if ...
677   3076  (1) weightbasedhealth: no, related phrases if ...
678    211  (1) weightbasedhealth: no, related phrases if ...
679   3969  (1) weightbasedhealth: no, related phrases if ...


Processing batch:   7%|▋         | 69/1000 [1:06:07<15:18:14, 59.18s/it]

     sm_id                                      answer_string
680   1041  (1) weightbasedhealth: no, related phrases if ...
681   2220  (1) weightbasedhealth: no, related phrases if ...
682    100  (1) weightbasedhealth: no, related phrases if ...
683   1858  (1) weightbasedhealth: no, related phrases if ...
684   3875  (1) weightbasedhealth: no, related phrases if ...
685   4498  (1) weightbasedhealth: no, related phrases if ...
686    392  (1) weightbasedhealth: no, related phrases if ...
687    600  (1) weightbasedhealth: no, related phrases if ...
688    190  (1) weightbasedhealth: no, related phrases if ...
689   1343  (1) weightbasedhealth: no, related phrases if ...


Processing batch:   7%|▋         | 70/1000 [1:07:05<14:51:26, 57.51s/it]

     sm_id                                      answer_string
690   2398  (1) weightbasedhealth: no, related phrases if ...
691   1917  (1) weightbasedhealth: no, related phrases if ...
692   1917  (1) weightbasedhealth: no, related phrases if ...
693   2204  (1) weightbasedhealth: no, related phrases if ...
694   2887  (1) weightbasedhealth: no, related phrases if ...
695    648  (1) weightbasedhealth: no, related phrases if ...
696   1120  (1) weightbasedhealth: no\n(2) weightstigma: y...
697   3115  (1) weightbasedhealth: no, related phrases if ...
698   4214  (1) weightbasedhealth: no, related phrases if ...
699    591  (1) weightbasedhealth: no, related phrases if ...


In [6]:
import pandas as pd

# File paths
file_paths = [
    "../../data2/answer_df_qwen1.csv",
    "../../data2/answer_df_qwen2.csv"
]

# Read and concatenate dataframes
answer_df_list = [pd.read_csv(file_path) for file_path in file_paths]
answer_df = pd.concat(answer_df_list, ignore_index=True)

# Save the concatenated dataframe
output_path = "../../data2/answer_df_qwen.csv"
answer_df.to_csv(output_path, index=False)
